In [ ]:
import openai
import pandas as pd
import gradio as gr

In [ ]:
openai.api_key = "824fe43e851f4862af326fa83c3d3cfe"

In [ ]:
file_path = './docs/Tweets.csv'
df = pd.read_csv(file_path)

In [ ]:
df.to_csv('./docs/Tweets_panda.csv', index=False)

In [9]:
!openai tools fine_tunes.prepare_data -f ./docs/Tweets_panda.csv


UnboundLocalError: local variable 'child' referenced before assignment

2560.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [ ]:
def create_training_file(file_path):
  file = openai.File.create(
      file=open(file_path, "rb"),
      purpose="fine-tune"
  )
  return file


training_file = create_training_file("/contents/DRIVE LOCATION/file_prepared.jsonl")
print(training_file)

In [ ]:
training_file['id']

In [ ]:
fine_tuned_model = openai.FineTune.create(training_file=training_file['id'],
model ='ada',
n_epochs=4)
print(fine_tuned_model)

In [ ]:
openai.FineTune.list_events(id="ft-YOUR FILE ABOVE") ['data']

In [ ]:
prompt_text = "TEST TWEET \n\n###\n\n"
response = openai.Completion.create(
    model="ada:YOUR MODEL",
    prompt=prompt_text,
    max_tokens=5,
    temperature=0
)
print(response['choices'][0]['text'])

In [ ]:
def generate_completion(user_prompt):
    hidden_context = " "
    prompt = hidden_context + user_prompt
    response = openai.Completion.create(
        model="ada:YOUR MODEL",
        prompt=prompt,
        max_tokens=10,
        temperature=0
    )
    return response.choices[0].text.strip()
iface = gr.Interface(fn=generate_completion, 
                     inputs=gr.inputs.Textbox(lines=5, placeholder='Enter a tweet...'), 
                     outputs='text',
                     title="Twitter Flight Happiness",
                     description="Based on thousands of tweets, determines if the tweet you entered is positive, negative, or nuetral",
                     input_labels="Tweet",
                     output_labels="Sentiment")

iface.launch(share=True)